In [1]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-265186d4-fd5d-4511-9567-d0a07da83296;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 368ms :: artifacts dl 14ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



Vorüberlegung, um Moderatoren zu bestimmen:
Moderatoren sind Nutzer, die in einem Zeitraum von 5 Minuten mehr als einen Pixel setzen konnten.

D.h.
SELECT d1.user_id

FROM dataFrame d1 JOIN dataFrame d2 ON d1.user_id = d2.user_id

WHERE d1.t > d2.t - 60 * 5 (60*5 entspricht 5 Minuten)

Wie setzt man das in dataFrames um???

<div style="color:lightblue">
(Sebastian):
schau mal den self join an den ich gemacht habe in 0.1-heuer-pixelklassen

Beispiel aus meinem Code:
```
selfjoinedFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
selfjoinedFrame = selfjoinedFrame.where('df2.t > df1.t').where('df2.t <= df1.t + 2*'+str(RZModel.max_t_dist))
selfjoinedFrame = selfjoinedFrame.where('Abs(df2.x - df1.x) <= 2*'+str(RZModel.max_xy_dist))
selfjoinedFrame = selfjoinedFrame.where('Abs(df2.y - df1.y) <= 2*'+str(RZModel.max_xy_dist))
selfjoinedFrame = selfjoinedFrame.select(F.col('df1.user_id'),F.col('df1.x').alias("x1"),F.col('df2.x').alias('x2'),F.col('df1.y').alias("y1"),F.col('df2.y').alias('y2'),F.col('df1.t').alias("t1"),F.col('df2.t').alias('t2'),F.col('df1.pixel_color').alias("c1"),F.col('df2.pixel_color').alias('c2'))
```

So kannst du das auch machen:
* df2.t > df1.t , dann ist das schonmal geordnet und du brauchst keine doppler wegen Reihenfolge betrachten
* df2.t < df1.t + (60*5) - eventuell 2 Sekunden grace period geben. Also (60*4 + 58) Echte Nutzer können durch Serverlag vlt ein paar Sekunden früher setzen - Moderatoren sind hier deutliche Ausreißer!
* dann diesen DataFrame wieder pro Nutzer aggregieren um zu gucken wie viele Pixel ein Nutzer anomal gesetzt hat :)

</div>

Bzw. eine zweite zu betrachtende Situation ist, dass Moderatoren auch mehrere Pixel auf einmal ändern können.

SELECT user_id, COUNT(x) --es reicht aus die x Koordinate zu zählen

FROM dataFrame

WHERE --Wie könnte die Where-Clause angegeben werden, dass nur ein bestimmter Zeitpunkt betrachtet wird?

HAVING COUNT(x) > 1

GROUP BY user_id;

<div style="color:lightblue">(Sebastian): Gruppieren nach t-Spalte und dann gucken wo count(t) > 1 ?</div>